# Mixture modeling with GloVe Embeddings

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.mixture
import scipy
import pandas as pd
from scipy import spatial
from sklearn.manifold import TSNE
import sklearn

#import code to generate data and do clustNP
from clustNP import clustNP, gauss_kernal_mat, gen_ZG, gen_C, gen_Ci, clustNP_obj, proj_simplex

from polyglot.mapping import Embedding

### Load 10d Embeddings

In [2]:
embeddings = Embedding.from_glove("glove10d.txt")

### Load Twitter Russian Trolls (run commented cells to recreate data matrix if desired)

In [3]:
# import pandas as pd
# import glob

# path = r'russian-troll-tweets-master' # use your path
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# frame.to_pickle('tweets_frame.pkl')

# to load
# frame = pd.read_pickle('tweets_frame.pkl')

In [4]:
# # tokenize, lemmatize
# import nltk

# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()

# def lemmatize_text(text):
#     return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

# df_en = frame.loc[frame['language'] == 'English']
# df_16 = df_en[df_en['publish_date'].str.contains('2016')]
# df_16['content']=df_16.content.apply(lemmatize_text)
# df_16_content = df_16['content']
# df_16.shape #878878

In [5]:
# df_16_content

In [6]:
# # pair words
# import random
# import string
# np.random.seed(0)
# punct = string.punctuation

# max_pairs = 5000000
# X = np.zeros((max_pairs,10)) #take 5mil word pairs max
# vs = np.zeros((280,10)) #preallocate max tweet char length array
# X_words = [] #store the words for each entry of X
# idx=0
# for j,twt in enumerate(df_16_content):
#     if np.mod(j,1000) == 0:
#         print(j)
#     l = len(twt)
#     random.shuffle(twt)
#     for i,word in enumerate(twt):
#         try:
#             word = word.strip(punct)
#             word = word.strip('#')
#             v = embeddings[word]
#             vs[i]  = v
#             X_words.append(word)
#             if np.sum(np.abs(v)) == 0:
#                 print(v)
#         except:
#             l-=1
#     l-=np.mod(l,2)
#     #stacking an even number of vectors from each tweet, can assign pairs ids after the fact
#     if l>0 and idx+l <= max_pairs:
#         X[idx:idx+l,:] = vs[:l]
#         idx+=l
#     elif l <= 0:
#         pass
#     else:
#         print("done")
#         break



In [7]:
##len(X_words) #3829919
##X_words = X_words[:-1] 
#print word pairs to a txt file to use with other methods
# lines = []
# for i in range(0,len(X_words)-1,2):
#     words = '%s %s\n'%(X_words[i], X_words[i+1])
#     lines.append(words)


In [8]:
# f = open("troll-corpus.txt", "w")
# f.writelines(lines)
# f.close()


# """
# to run java code:

# GPU-DMM: $ java -jar jar/STTM.jar -model GPUDMM -corpus dataset/troll-corpus.txt -vectors ../glove10d.txt -ntopics 10 -niters 1000 -name 'gpudmm'

# LF-DMM: $ java -jar jar/STTM.jar -model LFDMM -corpus dataset/troll-corpus.txt -vectors ../glove10d.txt -ntopics 10 -niters 1000 -name 'lfdmm'

# """

In [9]:
# # # remove extra rows
# # pair_ids = np.array([i for i in range(X.shape[0]//2) for j in range(2)])
# num_unfilled = np.sum(np.sum(np.abs(X),axis=1)==0) #1577844
# # X = X[:(max_pairs-num_unfilled),:]
# print(num_unfilled)

In [10]:
# # remove any cells that were filled with zeros
#np.where(np.sum(np.abs(X),axis=1)==0) # (array([ 3,  4,  5,  9, 10, 20, 26, 32]),)
# X = np.delete(X,[2,3,4,5,8,9,10,11,20,21,26,27,32,33],axis=0)


In [11]:
# # # # save
# # print(X.shape) #(3422142, 10)
# with open('tweets_preprocessed_lemmatized.npz', 'wb') as f:
# #     np.save(f, X)
#     np.savez_compressed(f, X)

# solve

### Original Param Init

In [14]:
with np.load('tweets_preprocessed_lemmatized.npz') as data:
    X = data["arr_0"]
pair_ids = np.array([i for i in range(X.shape[0]//2) for j in range(2)])

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.set(style="darkgrid")

# params
R = 200
best_ISE = 1e10;
d = X.shape[1]
M = 10
sd = np.mean(np.std(X, axis=0))
sigma = sd*(R)**(-1/(d+4)) #scott's rule of thumb
pair_ids = np.array([i for i in range(X.shape[0]//2) for j in range(2)])

In [16]:
#use random batch of 10000 pairs to initialize G,Z

# five seeds for five experiments
np.random.seed(0); results_filename = 'NDIGO_seed0';
#np.random.seed(1)
#np.random.seed(2)
# np.random.seed(3)


n_init = 10000
#n_init = X.shape[0]//2

rand_pairs = np.random.permutation(pair_ids.shape[0]//2)[:n_init]
idxs = 2*(np.maximum(rand_pairs-1, 0))
temp = np.append(np.insert(idxs, slice(1, None), 0), 0)
temp[1::2] = idxs+1
idxs = temp
Xmini = X[idxs,:]
rand_pair_ids = [pid for j in range(2) for pid in rand_pairs]
Z, G = gen_ZG(Xmini, R, sigma)

#Generate initialization
C = gen_C(Xmini, rand_pair_ids, Z, R, sigma)
# C = np.sum(Ci, axis=2)
# Cinv = np.linalg.inv(C)
# A, w, _ = scipy.sparse.linalg.svds(G@Cinv@G, k=M)

w, A = scipy.sparse.linalg.eigs(G, k=M)
w = np.real(w)
A = np.real(A)
for i in range(M):
    A[:,i] = proj_simplex(A[:,i])
#w = w/np.sum(w)
w = np.ones(M,) / M
A = A / np.sum(A,axis=0)
f, _ = clustNP_obj(A, w, G, C, n_init, 0)

best_ISE = f
best_sigma = sigma
best_R = R
best_w = w
best_Z = Z
best_C = C
best_G = G
best_A = A
print(best_sigma, best_R, best_ISE)

0.4484476088905496 200 0.018922095717083417


In [51]:
# with open('tweets_best_R_other_4.npy', 'wb') as f:
#     np.save(f, best_R)
# with open('tweets_best_w_other_4.npy', 'wb') as f:
#     np.save(f, best_w)
# with open('tweets_best_Z_other_4.npy', 'wb') as f:
#     np.save(f, best_Z)
# with open('tweets_best_G_other_4.npy', 'wb') as f:
#     np.save(f, best_G)
# with open('tweets_best_A_other_4.npy', 'wb') as f:
#     np.save(f, best_A)
# with open('tweets_best_sigma_other_4.npy', 'wb') as f:
#     np.save(f, best_sigma)

### Optimize

In [17]:
sigma = best_sigma
w0 = best_w
R = best_R
#w0 = np.array([0.5, 0.5])
Z = best_Z
G = best_G
A0 = best_A
#A0 = np.random.rand(R,M)
#A0 = A0 / np.sum(A0, axis=0)
ss=0.03

print(best_ISE)
# #solve
f_star, A_star, w_star, Z, n_iter = clustNP(X, pair_ids, A0, w0, Z, G, stepsize=ss, ss_decr=1, 
                                            epoch_decr = 40, method='psgd', max_iter=10, f_tol=1e-16, 
                                            grad_tol=1e-8, R=R, sigma=sigma, batch_size=1024, backtrack=False,
                                            decay=0.000001, momentum=0.2, large=True)
# f_star, A_star, w_star, Z, n_iter = clustNP(X, pair_ids, A_star, w_star, Z, G, stepsize=ss, ss_decr=1, 
#                                             epoch_decr = 40, method='psgd', max_iter=30, f_tol=1e-16, 
#                                             grad_tol=1e-8, R=R, sigma=sigma, batch_size=1024, backtrack=False,
#                                             decay=0.00001, momentum=0.2, large=True)
f_star

0.018922095717083417
processed observations this epoch:  512000
processed observations this epoch:  1024000
processed observations this epoch:  1536000
0.00040102995878448866
processed observations this epoch:  512000
processed observations this epoch:  1024000
processed observations this epoch:  1536000
0.00018490087865764848
processed observations this epoch:  512000
processed observations this epoch:  1024000
processed observations this epoch:  1536000
0.00013107076686919068
processed observations this epoch:  512000
processed observations this epoch:  1024000
processed observations this epoch:  1536000
0.00010901645263253318
processed observations this epoch:  512000
processed observations this epoch:  1024000
processed observations this epoch:  1536000
9.790277923335766e-05
processed observations this epoch:  512000
processed observations this epoch:  1024000
processed observations this epoch:  1536000
9.151861054979631e-05
processed observations this epoch:  512000
processed obse

8.121397403392084e-05

In [55]:
# with open('tweets_A_star_other_4.npy', 'wb') as f:
#     np.save(f, A_star)
# with open('tweets_w_star_other_4.npy', 'wb') as f:
#     np.save(f, w_star)
# with open('tweets_f_star_other_4.npy', 'wb') as f:
#     np.save(f, f_star)

In [ ]:
# A_star = np.load('tweets_A_star_other.npy')
# w_star = np.load('tweets_w_star_other.npy')
# A_star.shape

In [18]:
#make predictions
M=10
phats_train = np.zeros((X.shape[0],M))
for i in range(X.shape[0]):
    if np.mod(i, 10000) == 0:
        print(i)
    kx = gauss_kernal_mat(X[i, np.newaxis], best_Z, best_sigma)
    phats_train[i, :] = kx@A_star
ltr = np.zeros((X.shape[0], M))
ltr = w_star.T*phats_train
# decisions_train = np.argmax(ltr, 1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [19]:
# predicting individual training words
X_unique, idx_unique = np.unique(X, axis=0, return_index=True)
ltr_unique = ltr[idx_unique,:]
a = 20
top_words_topics = []
for m in range(M):
    top_a_idx = ltr_unique[:,m].argsort()[-a:][::-1]
    words = [embeddings.words[np.where((embeddings.vectors == tuple(X_unique[i,:])).all(axis=1))[0][0]] 
             for i in list(top_a_idx)]
    top_words_topics.append(' '.join(words) + '\n')
    print('topic %i :' %m, words)

topic 0 : ['yet', 'else', 'anyone', 'find', 'everyone', 'life', 'always', 'lot', 'exactly', 'none', 'certainly', 'sure', 'nothing', 'might', 'anything', 'fact', 'cut', 'wa', 'done', 'thought']
topic 1 : ['might', 'even', 'make', 'still', 'yet', 'get', 'could', 'good', 'going', 'life', 'right', 'lot', 'better', 'way', 'know', 'making', 'never', 'come', 'want', 'find']
topic 2 : ['return', 'ended', '12', 'win', '20', 'next', '15', '30', 'year', 'including', '10', 'ago', '11', '18', 'help', 'month', '16', '14', 'start', 'early']
topic 3 : ['support', 'become', 'terms', 'must', 'every', 'huge', 'related', 'problem', 'cannot', 'growth', 'particular', 'fact', 'special', 'changes', 'strong', 'reasons', 'details', 'involved', 'possible', 'politics']
topic 4 : ['say', 'want', 'go', 'know', 'going', 'good', 'world', 'believe', 'get', 'come', 'like', 'might', 'better', 'recent', 'hard', 'let', 'think', 'best', 'big', 'make']
topic 5 : ['say', 'want', 'news', 'wa', 'shit', 'university', 'quaalude'

In [21]:
fstr = "STTM-master/NDIGOresults/" + results_filename + ".topWords"
f = open(fstr, "w") # this can be used directly with the topic coherence score function in STTP
f.writelines(top_words_topics)
f.close()